# 5. Neural Networks

### *Table of Contents*

* 5.1 [Feed-forward Network Functions](#5.1-Feed-forward-Network-Functions)
* 5.2 [Network Training](#5.2-Network-Training)
    * 5.2.1 [Parameter optimization](#5.2.1-Parameter-optimization)
    * 5.2.2 [Local quadratic approximation](#5.2.2-Local-quadratic-approximation)
    * 5.2.4 [Gradient descent optimization](#5.2.4-Gradient-descent-optimization)
* 5.3 [Error Backpropagation](#5.3-Error-Backpropagation)

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Set random seed to make deterministic
np.random.seed(0)

# Ignore zero divisions and computation involving NaN values.
np.seterr(divide = 'ignore', invalid='ignore')

# Enable higher resolution plots
%config InlineBackend.figure_format = 'retina'

# Enable autoreload all modules before executing code
%load_ext autoreload
%autoreload 2

## 5.1 Feed-forward Network Functions

The linear models discussed in previous chapters are based on linear combinations of fixed (non)linear basis functions $\phi_j(\mathbf{x})$ and take the form,

$$
y(\mathbf{x},\mathbf{w}) = f\Bigg(\sum_{j=1}^M w_j\phi_j(\mathbf{x})\Bigg)
$$

where $f(\cdot)$ is a nonlinear activation function in the case of classification and the identity in the case of regression. Although such models have useful analytical properties, they are limited by the curse of dimentionality, and they need to adapt the basis functions to the data for large-scale problems. An alternative is to use a predefined number of basis functions but allow them to be adaptive during training. Thus, an extension of the model above is making the basis functions $\phi_j(\mathbf{x})$ depend on parameters and then adjust them along the coefficients $\{w_j\}$, during training.

*Neural networks* use basis functions that follow the same form, that is, each basis function is itself a nonlinear function of a linear combination of the inputs, where the coefficients are adapative parameters. Thus, the basic neural network model is described as a series of functional transformations.

1. Given the input variables $x_1,\dots,x_D$, we construct $M$ linear combinations in the form:

$$
a_j = \sum_{i=1}^D w_{ji}^{(1)}x_i + w_{j0}^{(1)}
$$

where $j=1,\dots,M$, and the superscript $(1)$ indicates the corresponding parameters of the *first* layer of the network. The input layer often uses the superscript $(0)$. The quantities $a_j$ are known as activations and each of them is transformed using a *differentiable* nonlinear activation function $h(\cdot)$ to give

$$
z_j = h(a_j)
$$

these correspond to the outputs of the basis functions, and in the context of neural networks are called *hidden units*. The following figure presents the two step computation of a single unit or neuron.

![Neuron](../images/neuron.png)

2. Following the same procedure, these output values from the *first* layer, are linearly combined again to give,

$$
a_k = \sum_{j=1}^M w_{ki}^{(2)}z_j + w_{k0}^{(2)}
$$

where $k=1,\dots,K$. This transformation corresponds to the *second* layer of the network. These output activations are transformed again using an appropriate activation function $h$ to give a set of outputs $y_k$. The following figure depicts the entire process for a $2$-layer network.

> It is called a $2$-layer neural network because there are two layers of adaptive weights.

![NeuralNet](../images/fg5_1.png)

The output unit activation function is determined by the nature of the data and the assumed distribution of target variables. Thus, for regression problems, the activation function can be the identity, so that $y_k=\alpha_k$, and for classification, the output uses a sigmoid, or softmax function, so that $y_k=\sigma(\alpha_k)$.

We may combine these stages to obtain the overall network function (using a sigmoid output unit), as follows,

$$
y_k(\mathbf{x},\mathbf{w}) = \sigma\Bigg(\sum_{j=1}^M w_{kj}^{(2)}h\Bigg(\sum_{i=1}^D w_{ji}^{(1)}x_i + w_{j0}^{(1)}\Bigg) + w_{k0}^{(2)} \Bigg)
$$

where biases on each layer can be absorbed into the set of weight parameters by defining an additional input variable $x_0=1$.

For efficiency reasons, in an actual implementation, its more convinient to represent the activations in some $\ell$-layer as an $M$-dimensional vector,

$$
\mathbf{a}^{(\ell)} = 
\begin{bmatrix}
a_1^{(\ell)} \\
a_2^{(\ell)} \\
\vdots \\
a_M^{(\ell)} 
\end{bmatrix} =
\begin{bmatrix}
\mathbf{w}_{1}^{(\ell)T}\mathbf{x} + w_{10}^{(\ell)} \\
\mathbf{w}_{2}^{(\ell)T}\mathbf{x} + w_{20}^{(\ell)} \\
\vdots \\
\mathbf{w}_{M}^{(\ell)T}\mathbf{x} + w_{M0}^{(\ell)} \\
\end{bmatrix} =
\begin{bmatrix}
\mathbf{w}_{1}^{(\ell)T} \\
\mathbf{w}_{2}^{(\ell)T} \\
\vdots \\
\mathbf{w}_{M}^{(\ell)T} \\
\end{bmatrix} \mathbf{x} + \mathbf{w}_{0}^{(\ell)} =
\mathbf{W}^{(\ell)}\mathbf{x} + \mathbf{w}_{0}^{(\ell)}
$$

In terms of matrix dimensions, we have $(M \times D) (D \times 1) + (M \times 1)$. Then, the activation function $h(\cdot)$ is applied on $\mathbf{a}^{(\ell)}$ to obtain,

$$
\mathbf{z}^{(\ell)} = h(\mathbf{a}^{(\ell)})
$$

**Notes:**

* The **key difference** compared to the perceptron is that the neural network uses continuous sigmoidal non-linear hidden units, whereas the perceptron uses step-function non-linearities. This means that the neural network function is differentiable which plays a central role in network training.


* If the activation functions of all hidden units are taken to be linear, then any such network can be replaced by a network without the hidden units. This follows from the fact that the composition of successive linear transformations is itself a linear transformation.

## 5.2 Network Training

###### Regression

A simple approach to the problem of determining the network parameters is to revisit the discussion of polynomial curve fitting, and attempt to minimize a sum-of-squares error function. Thus, given a training set comprising a set of input vectors $\{\mathbf{x}_n\}$, where $n=1,\dots,N$, and a corresponding set of target vectors $\{\mathbf{t}_n\}$, we minimize the error function,

$$
E(\mathbf{w}) = \frac{1}{2}\sum_{n=1}^N ||\mathbf{y}(\mathbf{x}_n, \mathbf{w}) - \mathbf{t}_n)||^2
$$

Consider regression problems, and for the moment, a single target variable $t$ that may take any real value. Similar to Section 3.1, we assume that $t$ follows a Gaussian distribution having an $\mathbf{x}$-dependent mean, which is given by the output of the neural network,

$$
p(t|\mathbf{x},\mathbf{w}) = \mathcal{N}(t|y(\mathbf{x},\mathbf{w}),\beta^{-1})
$$

For the above conditional distribution, it is sufficient to take the output unit activation function to be the identity, because such a network can approximate any continuous function from $\mathbf{x}$ to y. Given a data set of $N$ independent, identically distributed observations $\mathbf{X}$, and the corresponding target values $\mathsf{t}$, the likelihood function is as follows,

$$
p(\mathsf{t}|\mathbf{X},\mathbf{w},\beta) = \prod_{n=1}^N p(t_n|\mathbf{x}_n,\mathbf{w},\beta) = \prod_{n=1}^N \mathcal{N}(t_n|y(\mathbf{x}_n,\mathbf{w}),\beta^{-1})
$$

Then, by taking the negative logarithm, we obtain the same error function derived in (3.11). By minimizing the error function, we obtain the maximum likelihood solution $\mathbf{w}_{ML}$. Having found $\mathbf{w}_{ML}$, the value of $\beta$ can be found using (3.21), derived by minimizing the negative log likelihood.

> **IMPORTANT:** Keep in mind, however, that the nonlinearity of the network function $y(\mathbf{x}_n,\mathbf{w})$ causes the error to be nonconvex, and so a local maxima of the likelihood may be found, corresponding to local minima of the error function.

###### Classification

In binary classification a single target variable $t$ such that $t=1$ for class $\mathcal{C}_1$ and $t=0$ for class $\mathcal{C}_2$. We consider a network having a single output whose activation function is a logistic sigmoid,

$$
y = \sigma(\alpha) = \frac{1}{1 + \exp(-\alpha)}
$$

so that $0 \leq y(\mathbf{x},\mathbf{w}) \leq 1$, is interpreted as the conditional probability $p(\mathcal{C}_k|\mathbf{x})$ given by $1 - y(\mathbf{x},\mathbf{w})$. The conditional distribution of targets given inputs is then a Bernoulli distribution of the form,

$$
p(t|\mathbf{x},\mathbf{w}) = y(\mathbf{x},\mathbf{w})^t\{1-y(\mathbf{x},\mathbf{w})\}^{1-t}
$$

Given a training set of independent observations, the the error function by the negative log-likelihood, if the *cross-entropy* error function of the form,

$$
E(\mathbf{w}) = -\sum_{n=1}^N \{t_n\ln y_n + (1-t_n)\ln(1-y_n)\}
$$

> **NOTE**: There is no analogue of the noise precision $\beta$ because the target values are assumed to be correctly labelled.

> **IMPORTANT**: There is a natural choice of both output unit activation function and matching error function, according to the type of problem being solved. For regression we rely on linear outputs and sum-of-squares error, and for binary logistic sigmoid (binary) or softmax (multiclass) outputs and cross-entropy error function.

### 5.2.1 Parameter optimization

The goal is to find a vector such that $E(\mathbf{w})$ is minimized. However, the error function has a highly nonlinear dependence on the weights, and so there are many points in the weight space at which the gradient vanishes. Since there is no hope of finding an analytical solution to the equation $\nabla E(\mathbf{w})=0$ we resort to iterative numerical procedures. Most of these techniques involve choosing an initial value $\mathbf{w}^{(0)}$ for the weight vector and then moving through weight space in a succession of steps of the form,

$$
\mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} + \Delta\mathbf{w}^{(\tau)}
$$

Such algorithms involve different choices for the weight vector update $\Delta\mathbf{w}^{(\tau)}$. Usually, they make use of gradient information and therefore require that, after each update, the value of $\nabla E(\mathbf{w})$ is evaluated at the updated weight vector $\mathbf{w}^{(\tau+1)}$. In order to understand the importance of gradient information, it is useful to consider a local approximation to the error function based on a Taylor expansion.

### 5.2.2 Local quadratic approximation

Consider the second-order Taylor expansion of $E(\mathbf{w})$ around some point $\hat{\mathbf{w}}$,

$$
E(\mathbf{w}) \approx E(\hat{\mathbf{w}}) +(\mathbf{w} - \hat{\mathbf{w}})^T\mathbf{b} + \frac{1}{2}(\mathbf{w} - \hat{\mathbf{w}})^T\mathbf{H}(\mathbf{w} - \hat{\mathbf{w}})
$$

where $\mathbf{b}=\nabla E|_{\mathbf{w}=\hat{\mathbf{w}}}$ and $\mathbf{H}=\nabla\nabla E$ is the Hessian matrix of second derivatives. Thus, the local approximation to the gradient is given by,

$$
E(\mathbf{w}) \approx \mathbf{b} + \mathbf{H}(\mathbf{w} - \hat{\mathbf{w}})
$$

for points $\mathbf{w}$ that are sufficiently close to $\hat{\mathbf{w}}$, these expressions give reasonable approximations for the error and its gradient.

### 5.2.4 Gradient descent optimization

The simplest approach to using gradient information is to choose the weight update to be a small step in the direction of the negative gradient, so that,

$$
\mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} - \eta\nabla E(\mathbf{w}^{(\tau)})
$$

where the parameter $\eta > 0$ is known as *learning rate*. At each step the weight vector is moved in the direction of the greatest rate of decrease of the error function, and so the approach is known as *gradient descent* or *steepest descent*. Note that the error function is defined with respect to a training set, and thus, each step requires that the entire training set be processed in order to evaluate $\nabla E$. Techniques that use the whole data set at once are called *batch* methods. Although such an approach might intuitively seem reasonable, in fact it turns out to be a poor algorithm.

For *batch optimization*, there are more efficient methods, such as conjugate gradients and quasi-Newton methods, which are much more robust and much faster than simple gradient descent. Unlike gradient descent, these algorithms have the property that the error function always decreases at each iteration unless the weight vector has arrived at a local or global minimum.

There is, however, an on-line version of gradient descent, known as *sequential gradient descent* or *stochastic gradient descent*, that has proved useful in practice for training neural networks on large data sets. Error functions based on maximum likelihood for a set of independent observations comprise a sum of terms, one for each data point,

$$
E(\mathbf{w}) = \sum_{n=1}^N E_n(\mathbf{w})
$$

Stochastic gradient descent makes an update to the weight vector based on **one data point** at a time, so that

$$
\mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} - \eta\nabla E_n(\mathbf{w}^{(\tau)})
$$

The update is repeated by cycling through the data either in sequence or by selecting points at random with replacement. There are of course intermediate scenarios in which the updates are based on batches of data points. One advantage of on-line methods compared to batch methods is that the former handle redundancy in the data much more efficiently. To see, this consider an extreme example in which we take a data set and double its size by duplicating every data point. Note that this simply multiplies the error function by a factor of $2$ and so is equivalent to using the original error function. Batch methods will require double the computational effort to evaluate the batch error function gradient, whereas online methods will be unaffected. Another property of on-line gradient descent is the possibility of escaping from local minima, since a stationary point with respect to the error function for the whole data set will generally not be a stationary point for each data point individually.

## 5.3 Error Backpropagation

Error backpropagation, or simply backprop, is an efficient technique for evaluating the gradient of an error function $E(\mathbf{w})$ for a feed-forward neural network. This can be achieved using a local message passing scheme in which information is sent alternately forwards and backwards through the network.

Most training algorithms involve an iterative procedure for minimization of an error function. At each such step, we can distinguish between two stages:

1. The derivatives of the error function must be evaluated. The important contribution of the backpropagation technique is in providing a computationally efficient method for evaluating such derivatives. Since the errors are propagated backwards through the network, we use the term backpropagation to describe the evaluation of derivatives. 


2. The derivatives are then used to compute the adjustments to be made to the weights, e.g., gradient descent. 

It is important to recognize that these stages are distinct. Thus, the first stage, namely the propagation of errors backwards through the network in order to evaluate derivatives, can be applied to many other kinds of networks and not just the multilayer perceptron. It can also be applied to error functions other than the simple sum-of-squares, and to the evaluation of other derivatives, such as, the Jacobian and Hessian matrices.

### 5.3.1 Evaluation of error-function derivatives

We derive the backpropagation algorithm for a general network of arbitrary topology and arbitraty non-linear activation functions, and a broad class of error function.

Many error functions of practical interest comprise a sum of terms, one of each data point in the training set, so that,

$$
E(\mathbf{w}) = \sum_{n=1}^N E_n(\mathbf{w})
$$

Here, we consider the evalution of a single term $\nabla E_n(\mathbf{w})$. Consider the first linear model, where the outputs are linear combinations of the input variables.

$$
y_{nk} = y_k(\mathbf{x}_n, \mathbf{w}) = \sum_i w_{ki}x_i
$$

thus, the error function for input example $n$, takes the form,

$$
E_n = \frac{1}{2} \sum_k(y_{nk} - t_{nk})^2
$$

and its gradient with respect to $w_{ji}$, is given by,

$$
\frac{\partial E_n}{\partial w_{ji}} = (y_{nj} - t_{nj})x_{ni}
$$

In a general feed-forward network, each unit computes a weighted sum of the inputs, as follows,

$$
a_j = \sum_i  w_{ji}z_i
$$

where $z_i$ is the activation of a unit, in the previous layer, that sends a connection to unit $j$, and $w_{ji}$ is the weight associated with the connection. Then, the sum $a_j$ is transformed by a non-linear activation function $h(\cdot)$ to give $z_j$ in the form,

$$
z_j = h(a_j)
$$

In turn, $z_j$ may be sent as a connection to a subsequent unit in order to participate in another activation. In order to apply backprop, we assume that we have computed the activations of all hidden and output units in the network, a process called *forward propagation* because it may be regarded as the forward flow of information through the network.

Conside again the evaluation of the derivative $E_n$. Given an arbitraty unit $j$, $E_n$ depends on the weight $w_{ji}$ only via the input $a_j$. Therefore, according to the chain rule for partial derivatives, we obtain,

$$
\frac{\partial E_n}{\partial w_{ji}} = \frac{\partial E_n}{\partial a_j} \frac{\partial a_j}{\partial w_{ji}} \overset{(5.48)}= \frac{\partial E_n}{\partial a_j} z_i
$$

Since $z_i$ is computed during forward propagation, we only need to compute $\frac{\partial E_n}{\partial a_j}$. For simplicity, lets define $\delta_j = \frac{\partial E_n}{\partial a_j}$.

For the output units, we have that $\delta_k = \frac{\partial E_n}{\partial y_k} \overset{(5.46)}= y_k - t_k$.

For hidden units, we apply again the chain rule,

$$
\begin{align}
\delta_j &= \frac{\partial E_n}{\partial a_j} = \sum_k \frac{\partial E_n}{\partial a_k} \frac{\partial a_k}{\partial a_j} \\ 
&= \sum_k \delta_k \frac{\partial a_k}{\partial a_j} \overset{(5.48)}= \sum_k \delta_k \Big( \frac{\partial}{\partial a_j} \sum_j w_{kj} z_j \Big) \overset{(5.49)}= \sum_k \delta_k \Big( \frac{\partial}{\partial a_j} \sum_j w_{kj} h(a_j) \Big) \\
&= h'(a_j) \sum_k w_{kj} \delta_k
\end{align}
$$

where the sum runs over all $k$ units to which $j$ sends connections. Thus, for a particular hidden unit, $\delta$ is obtained by propagating the $\delta$ backwards from units higher up in the network.

![Backprop](../images/fg5_7.png)

Since, we know the $\delta$ for the output units, we can recersively evaluate the $\delta$ for all the hidden units in a feed-forward neural net, regardless of the topology.

**Error Backpropagation algorithm**

1. Forward propagate any input vector $\mathbf{x}_n$ using $(5.48)$ and $(5.49)$.

2. Evalute $\delta_k$ for output units using $(5.54)$.

3. Backpropagate $\delta$ using $(5.56)$ to obtain $\delta_j$ for each hidden unit.

4. Evalute the required derivatives using $(5.53)$.

For batch methods, the derivative of the total error $E$ is obtained be repeating the above steps for each example $n$ and then summing over all examples.